In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

### Window State Function

In [2]:
## heavyside function
def h(t):
    if t <= 0:
        return 0
    if t > 0:
        return 1

In [3]:
window = np.zeros((2, 2), dtype=int)

a_star = lambda a11, a12, a21, a22: h(h(a11+a12+a22-1) + h(a12+a21-1))

def element_state(window, h):
    a11 = window[0][0]
    a12 = window[0][1]
    a21 = window[1][0]
    a22 = window[1][1]
    return a_star(a11, a12, a21, a22)

The changes in each combination of bits in a 2x2 window

In [4]:
arr = np.empty(4, dtype=int)

for i in range (16):
    bin_val = format(i, '04b')
    for index, bit in enumerate(bin_val):
        arr[index]  = int(bit)
    window[0][0] = arr[0]
    window[0][1] = arr[1]
    window[1][0] = arr[2]
    window[1][1] = arr[3]
    state = element_state(window, h)
    if (window[0][1] != state):
        print("\nWhen the window is :\n", window, "\nThe upper right will change from", window[0][1], "to", state)



When the window is :
 [[0 1]
 [0 0]] 
The upper right will change from 1 to 0

When the window is :
 [[1 0]
 [0 1]] 
The upper right will change from 0 to 1

When the window is :
 [[1 0]
 [1 1]] 
The upper right will change from 0 to 1


The phi state function in action on an arbitrary matrix

In [5]:
def phi_single(window, size, i, j, h):
    if (i+1 < size  and j-1 >= 0):
        a11 = window[i][j-1]
        a12 = window[i][j]
        a21 = window[i+1][j-1]
        a22 = window[i+1][j]
    elif (i+1 >= size and j+1 < 0):
        a11 = 0
        a12 = window[i][j]
        a21 = 0
        a22 = 0
    elif (i+1 >= size):
        a11 = window[i][j-1]
        a12 = window[i][j]
        a21 = 0
        a22 = 0
    elif (j-1 < 0):
        a11 = 0
        a12 = window[i][j]
        a21 = 0
        a22 = window[i+1][j]
    else:
        assert False, "Something is wrong with the matrix"
    window[i][j] = a_star(a11, a12, a21, a22)

In [6]:
arbt = np.zeros((4, 4), dtype=int)

arbt[1][1] = 1
arbt[2][2] = 1
print("Matrix before:\n", arbt)

phi_single(arbt, 4, 1, 2, h)
print("\nMatrix after:\n", arbt)

Matrix before:
 [[0 0 0 0]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 0 0]]

Matrix after:
 [[0 0 0 0]
 [0 1 1 0]
 [0 0 1 0]
 [0 0 0 0]]


### Parallelism in action

In [7]:

from joblib import Parallel, delayed

In [8]:
def phi(window, size, i, j, h):
    if (i+1 < size  and j-1 >= 0):
        a11 = window[i][j-1]
        a12 = window[i][j]
        a21 = window[i+1][j-1]
        a22 = window[i+1][j]
    elif (i+1 >= size and j+1 < 0):
        a11 = 0
        a12 = window[i][j]
        a21 = 0
        a22 = 0
    elif (i+1 >= size):
        a11 = window[i][j-1]
        a12 = window[i][j]
        a21 = 0
        a22 = 0
    elif (j-1 < 0):
        a11 = 0
        a12 = window[i][j]
        a21 = 0
        a22 = window[i+1][j]
    else:
        assert False, "Something is wrong with the matrix"
    return a_star(a11, a12, a21, a22)


def change_elements(elements_to_operate_on, result):
    for index, element in enumerate(elements_to_operate_on):
        i = element[2]
        j = element[3]
        matrix = element[0]
        matrix[i][j] = result[index]

Proof that a change of '1' to '0' won't disconnect the connected components

In [9]:
m1 = np.zeros((3, 3), dtype=int)

m1[0][0] = 1
m1[1][1] = 1
m1[2][2] = 1

print("Matrix before:\n", m1)

elements_to_operate_on = [
    (m1, 3, 0, 1, h),
    (m1, 3, 1, 1, h),
    (m1, 3, 1, 2, h)
]

result = Parallel(n_jobs=-1)(delayed(phi)(*element) for element in elements_to_operate_on)

change_elements(elements_to_operate_on, result)

print("\nMatrix after:\n", m1)

Matrix before:
 [[1 0 0]
 [0 1 0]
 [0 0 1]]

Matrix after:
 [[1 1 0]
 [0 0 1]
 [0 0 1]]


Proof that a change of '0' to '1' won't merge connected components

In [10]:
m2 = np.zeros((4, 4), dtype=int)

m2[0][1] = 1
m2[0][2] = 1
m2[0][3] = 1
m2[1][2] = 1
m2[1][3] = 1
m2[2][0] = 1
m2[2][3] = 1
m2[3][0] = 1
m2[3][1] = 1

print("Matrix before:\n", m2)

elements_to_operate_on = [
    (m2, 4, 2, 1, h),
    (m2, 4, 1, 2, h)
]

result = Parallel(n_jobs=-1)(delayed(phi)(*element) for element in elements_to_operate_on)

change_elements(elements_to_operate_on, result)

print("\nMatrix after:\n", m2)

Matrix before:
 [[0 1 1 1]
 [0 0 1 1]
 [1 0 0 1]
 [1 1 0 0]]

Matrix after:
 [[0 1 1 1]
 [0 0 0 1]
 [1 1 0 1]
 [1 1 0 0]]


Algorithm on a singly connected component

In [38]:
m3 = np.zeros((6, 6), dtype=int)

m3[0][0] = 1
m3[0][1] = 1
m3[0][2] = 1
m3[1][1] = 1
m3[1][2] = 1
m3[2][1] = 1
m3[2][2] = 1
m3[3][0] = 1
m3[3][1] = 1
m3[3][2] = 1
m3[3][3] = 1
m3[3][4] = 1
m3[3][5] = 1
m3[4][1] = 1
m3[4][2] = 1
m3[4][3] = 1
m3[4][4] = 1
m3[4][5] = 1
m3[5][2] = 1
m3[5][3] = 1

print("Matrix before:\n", m3)

elements_to_operate_on = [(m3, 6, i, j, h) for i in range(6) for j in range(6)]

for i in range(9):
    result = Parallel(n_jobs=-1)(delayed(phi)(*element) for element in elements_to_operate_on)
    change_elements(elements_to_operate_on, result)
    print("\nMatrix after", i+1, "iterrations:\n", m3)

Matrix before:
 [[1 1 1 0 0 0]
 [0 1 1 0 0 0]
 [0 1 1 0 0 0]
 [1 1 1 1 1 1]
 [0 1 1 1 1 1]
 [0 0 1 1 0 0]]

Matrix after 1 iterrations:
 [[0 1 1 0 0 0]
 [0 1 1 0 0 0]
 [0 1 1 1 0 0]
 [0 1 1 1 1 1]
 [0 0 1 1 1 1]
 [0 0 0 1 0 0]]

Matrix after 2 iterrations:
 [[0 1 1 0 0 0]
 [0 1 1 1 0 0]
 [0 1 1 1 1 0]
 [0 0 1 1 1 1]
 [0 0 0 1 1 1]
 [0 0 0 0 0 0]]

Matrix after 3 iterrations:
 [[0 1 1 1 0 0]
 [0 1 1 1 1 0]
 [0 0 1 1 1 1]
 [0 0 0 1 1 1]
 [0 0 0 0 1 1]
 [0 0 0 0 0 0]]

Matrix after 4 iterrations:
 [[0 1 1 1 1 0]
 [0 0 1 1 1 1]
 [0 0 0 1 1 1]
 [0 0 0 0 1 1]
 [0 0 0 0 0 1]
 [0 0 0 0 0 0]]

Matrix after 5 iterrations:
 [[0 0 1 1 1 1]
 [0 0 0 1 1 1]
 [0 0 0 0 1 1]
 [0 0 0 0 0 1]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]]

Matrix after 6 iterrations:
 [[0 0 0 1 1 1]
 [0 0 0 0 1 1]
 [0 0 0 0 0 1]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]]

Matrix after 7 iterrations:
 [[0 0 0 0 1 1]
 [0 0 0 0 0 1]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]]

Matrix after 8 iterrations:
 [[0 0 0 0 0 1]
 

Algorithm on a multiply connected component

In [12]:
m4 = np.zeros((8, 8), dtype=int)

m4[0][0] = 1
m4[0][1] = 1
m4[0][2] = 1

m4[1][1] = 1
m4[1][2] = 1
m4[1][3] = 1
m4[1][4] = 1
m4[1][5] = 1

m4[2][1] = 1
m4[2][2] = 1
m4[2][5] = 1
m4[2][6] = 1

m4[3][0] = 1
m4[3][1] = 1
m4[3][2] = 1
m4[3][3] = 1
m4[3][4] = 1
m4[3][5] = 1
m4[3][6] = 1
m4[3][7] = 1

m4[4][0] = 1
m4[4][1] = 1
m4[4][4] = 1
m4[4][7] = 1

m4[5][1] = 1
m4[5][2] = 1
m4[5][3] = 1
m4[5][4] = 1
m4[5][7] = 1

m4[6][3] = 1
m4[6][4] = 1
m4[6][5] = 1
m4[6][6] = 1
m4[6][7] = 1

m4[7][2] = 1
m4[7][3] = 1

print("Matrix before:\n", m4)

elements_to_operate_on = [(m4, 8, i, j, h) for i in range(8) for j in range(8)]

for i in range(13):
    result = Parallel(n_jobs=-1)(delayed(phi)(*element) for element in elements_to_operate_on)
    change_elements(elements_to_operate_on, result)
    print("\nMatrix after", i+1, "iterrations:\n", m4)

Matrix before:
 [[1 1 1 0 0 0 0 0]
 [0 1 1 1 1 1 0 0]
 [0 1 1 0 0 1 1 0]
 [1 1 1 1 1 1 1 1]
 [1 1 0 0 1 0 0 1]
 [0 1 1 1 1 0 0 1]
 [0 0 0 1 1 1 1 1]
 [0 0 1 1 0 0 0 0]]

Matrix after 1 iterrations:
 [[0 1 1 1 0 0 0 0]
 [0 1 1 1 1 1 1 0]
 [0 1 1 1 0 1 1 1]
 [1 1 1 1 1 1 1 1]
 [0 1 1 0 1 0 0 1]
 [0 0 1 1 1 1 0 1]
 [0 0 0 1 1 1 1 1]
 [0 0 0 1 0 0 0 0]]

Matrix after 2 iterrations:
 [[0 1 1 1 1 0 0 0]
 [0 1 1 1 1 1 1 1]
 [0 1 1 1 1 1 1 1]
 [0 1 1 1 1 1 1 1]
 [0 0 1 1 1 1 0 1]
 [0 0 0 1 1 1 1 1]
 [0 0 0 1 1 1 1 1]
 [0 0 0 0 0 0 0 0]]

Matrix after 3 iterrations:
 [[0 1 1 1 1 1 0 0]
 [0 1 1 1 1 1 1 1]
 [0 1 1 1 1 1 1 1]
 [0 0 1 1 1 1 1 1]
 [0 0 0 1 1 1 1 1]
 [0 0 0 1 1 1 1 1]
 [0 0 0 0 1 1 1 1]
 [0 0 0 0 0 0 0 0]]

Matrix after 4 iterrations:
 [[0 1 1 1 1 1 1 0]
 [0 1 1 1 1 1 1 1]
 [0 0 1 1 1 1 1 1]
 [0 0 0 1 1 1 1 1]
 [0 0 0 1 1 1 1 1]
 [0 0 0 0 1 1 1 1]
 [0 0 0 0 0 1 1 1]
 [0 0 0 0 0 0 0 0]]

Matrix after 5 iterrations:
 [[0 1 1 1 1 1 1 1]
 [0 0 1 1 1 1 1 1]
 [0 0 0 1 1 1 1 1]
 [0 0 0 1 1 

### Counting connected components

In [36]:
def phi_cnt(window, size, i, j):
    if (i+1 < size  and j-1 >= 0):
        a11 = window[i][j-1]
        a12 = window[i][j]
        a21 = window[i+1][j-1]
        a22 = window[i+1][j]
    elif (i+1 >= size and j+1 < 0):
        a11 = 0
        a12 = window[i][j]
        a21 = 0
        a22 = 0
    elif (i+1 >= size):
        a11 = window[i][j-1]
        a12 = window[i][j]
        a21 = 0
        a22 = 0
    elif (j-1 < 0):
        a11 = 0
        a12 = window[i][j]
        a21 = 0
        a22 = window[i+1][j]
    else:
        assert False, "Something is wrong with the matrix"
    new_state = a_star(a11, a12, a21, a22)
    counter = 0
    if new_state == 0:
        if ((i-1 < 0 or window[i-1][j] == 0) and ((i-1 < 0 and j+1 >= size) or window[i-1][j+1] == 0) and (j+1 >= size or window[i][j+1] == 0)):
            counter = 1
    return new_state, counter

In [41]:
print("Matrix before:\n", m4)

elements_to_operate_on = [(m4, 8, i, j) for i in range(8) for j in range(8)]
count = 0 
for i in range(13):
    results = Parallel(n_jobs=-1)(delayed(phi_cnt)(*element) for element in elements_to_operate_on)
    result = [r[0] for r in results]
    add = [r[1] for r in results]
    change_elements(elements_to_operate_on, result)
    print("\nMatrix after", i+1, "iterrations:\n", m4)
print("\nNumber of connected components:", count)

Matrix before:
 [[1 1 1 0 0 0 0 0]
 [0 1 1 1 1 1 0 0]
 [0 1 1 0 0 1 1 0]
 [1 1 1 1 1 1 1 1]
 [1 1 0 0 1 0 0 1]
 [0 1 1 1 1 0 0 1]
 [0 0 0 1 1 1 1 1]
 [0 0 1 1 0 0 0 0]]


IndexError: index 8 is out of bounds for axis 0 with size 8